#  Проверка данных на полноту

Импортируем необходимые для расчетов библиотеки

In [1]:
import pandas as pd
import numpy as np
import re

Прочитаем данные и присвоим из датафрейму, выведем первые 5 строк данных.

In [2]:
Cashback = pd.read_csv('Case Data.csv')
Cashback.head(5)

,﻿id чека,Дата и время,ИНН,IDПользователя,Товар,Бренд,Теги,Цена,Кол-во,Сумма,Кэшбэк
0,0000a622f9c8f1821ca549fbcfd47405,21.12.2019 15:15,2310031475,2700449a73d4cb9d8e909c706bf16fbe,libresse natural care ultra прокладки normal 10ш,libresse,личная гигиена; красота и здоровье; прокладки,63.2,2.0,126.4,NaN
1,0000a622f9c8f1821ca549fbcfd47405,21.12.2019 15:15,2310031475,2700449a73d4cb9d8e909c706bf16fbe,pampers трусики-подг 5 junior 12-17/15-18кг 96шт,pampers,NaN,1549.0,1.0,1549.0,154.9
2,0000a622f9c8f1821ca549fbcfd47405,21.12.2019 15:15,2310031475,2700449a73d4cb9d8e909c706bf16fbe,salton губка/гл кож с дозатор(чер)(шанхай продак,NaN,NaN,135.2,1.0,135.2,NaN
3,0000a622f9c8f1821ca549fbcfd47405,21.12.2019 15:15,2310031475,2700449a73d4cb9d8e909c706bf16fbe,soft ватные палочки 200шт пластиковый стакан(кот,NaN,NaN,31.2,1.0,31.2,NaN
4,0000a622f9c8f1821ca549fbcfd47405,21.12.2019 15:15,2310031475,2700449a73d4cb9d8e909c706bf16fbe,моя цена/степной тюльпан туал/бум без вт(бумагар,NaN,NaN,4.9,1.0,4.9,NaN


Посмотрим в каких стобцах отсутствуют значения с помощью функции isna()

In [17]:
Cashback.isna().sum(axis=0)

﻿id чека               0
Дата и время           0
ИНН                    0
IDПользователя         0
Товар                  0
Бренд             178495
Теги              128391
Цена                  52
Кол-во                52
Сумма                 52
Кэшбэк            253552
dtype: int64

Значения отсутствуют в 6 столбцах из 11.
Составим таблицу, в которой будет отражено сколько значений отсутствует в каждом из столбцов.

In [3]:
Cashback_Nan = pd.concat([pd.DataFrame([[ Cashback[Cashback.columns[i]].isna().sum(),Cashback.columns[i]]], 
                                       columns={'Nan Quantity','Column'}) for i in range(5,11)])
Cashback_Nan.set_index('Column', inplace=True)
%time
Cashback_Nan


Wall time: 0 ns


,Nan Quantity
Column,
178495,Бренд
128391,Теги
52,Цена
52,Кол-во
52,Сумма
253552,Кэшбэк


Больше всего пропущщенных значений имеет колонка Кэшбэк, такэе большое количество пропущенных значений имеют поля Бренд и Теги

# Заполнение пропущенных строк

Заполним столбец "Бренд". Для этого сначала отберем строки в которых отсутствуют значения и будем работать только с ним так как процессы обработки строковых полей занимают много времени и ресурсов:

In [4]:
NansIndex = np.array(Cashback.query('Бренд != Бренд')['Бренд'].index)

Если посмотреть на данные станет заметно, что в строках с отсутвующим значением в столбце "Бренд" в столбце "Товар" часто присутствует название бренда этого товара и он есть в столбце "Бренд" других строк. Следовательно, можно пройтись по столбцу "Товар" и если присутствует сочетание символов как в одном из полей столбца "Бренд" других строк, то присвоить это сочетание в поле "Бренд" того товара, где оно отсутствут.

Выполним данную задачу с помощью метода unique(), так мы найдем название брендов, которые уже присутствуют в датасете, и функции search() модуля re, так мы будем искать в ячейках столбца "Товар" слова, которые есть в списке Брендов.

In [5]:
listNan = '|'.join(list(map(str,Cashback['Бренд'].unique()))) # ищем уникальные значения Брендов и объединяем через |,
#что будет работать как ИЛИ при использовании функции search

In [8]:
CheckPattern = '(?:{})'.format(listNan)
BrendName = Cashback.loc[:,'Бренд']  # пока создадим отдельную Серию для удобного просмотра, что выйдет при подстановке
for i in NansIndex:
    if bool(re.search(CheckPattern, Cashback.loc[i,'Товар'], flags=re.I)):
        BrendName[i] = re.search(CheckPattern, Cashback.loc[i,'Товар'], flags=re.I).group(0)
        print(BrendName[i])#вывод при работе цикла, чтобы контролировать работу в "реальном времени"

<ipython-input-8-7a306012ced4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BrendName[i] = re.search(CheckPattern, Cashback.loc[i,'Товар'], flags=re.I).group(0)


я
e
магнит
я
hoegaarden
я
тема
тема
e
maybelline
мое солнышко
e
я
taft
я
я
я
я
я
я
аленка
я
я
я
российский
e
я
whiskas
я
я
я
глобус
я
чудо
я
актимель
актимель
я
глобус
я
я
fa
ной
я
я
воздушный
я
я
я
я
лента
я
я
я
я
актимель
растишка
мираторг
мираторг
принцесса нури
я
ной
я
e
e
e
я
чистая линия
glade
e
я
я
e
я
e
e
e
дядя ваня
aro
aro
parmalat
e
я
e
я
aro
я
я
дядя ваня
я
rocs
я
я
я
e
троекурово
я
я
я
я
la fresh
я
я
ной
магнит
я
zewa
maggi
e
я
я
тысяча озер
e
e
я
я
starbucks
e
dreamies
guinness
miller
e
я
я
я
ной
я
русская валюта
стопарик
я
я
любятово
friskies
friskies
friskies
maggi
maggi
я
я
я
я
магнит
я
e
я
e
я
я
российский
papia
миф
perfect fit
лента
airwick
e
kamis
kamis
ной
я
магнит
я
zewa
я
e
аленка
я
магнит
я
смешарики
смешарики
смешарики
смешарики
тема
тема
хлебный дом
juicy fruit
белебеевский
коровка из кореновки
магнит
я
чистая линия
e
ласка
dove
я
e
librederm
я
365
барни
e
я
аморе
я
e
vernel
я
я
я
магнит
я
я
я
я
я
я
я
gliss kur
чистая линия
aos
la fresh
e
я
магнит
я
я
e
чистая

e
я
bella
la fresh
я
arko
aura
friskies
friskies
friskies
friskies
ной
магнит
агрокомплекс
bonfesto
bonfesto
camay
dove
greenfield
ной
я
город сыра
я
дядя ваня
я
я
я
я
московский провансаль
я
магнит
столичная
я
фрутоняня
фрутоняня
фрутоняня
фрутоняня
я
я
я
garnier
e
я
e
l'oreal
palmolive
president
простоквашино
jardin
байсад
лента
лента
лента
я
e
e
palmolive
e
я
я
я
я
hochland
palette
e
zewa
я
e
я
помидорка
ной
я
магнит
я
e
мистер лис
dove
e
e
ной
juicy fruit
maggi
академия комфорта
я
я
магнит
радость вкуса
я
я
я
garnier
e
dove
persil
deonica
e
gourmet
gourmet
perfect fit
я
я
e
я
e
я
я
я
я
я
я
e
я
я
arla
я
брест-литовск
брест-литовск
e
я
я
e
e
gold mine beer
j7
балтийский берег
чешское
я
я
ной
авида
я
авида
я
я
maggi
я
агуша
я
махеев
магнит
ряба
простоквашино
коровка из кореновки
я
папа может
e
e
e
я
я
bref
glade
la fresh
я
я
ной
я
я
e
ласка
papia
e
papia
ной
global village
красная цена
zewa
la fresh
ной
я
я
arko
la fresh
e
я
я
я
e
дубки
я
россия
ной
глобус
арарат
я
я
я
я
глобус
ld
я
г

e
e
я
лента
e
цезарь
lipton
e
e
я
e
viola
viola
я
я
я
я
я
я
я
liebendorf
papia
papia
я
я
магнит
я
я
я
dove
dove
e
e
я
я
я
я
я
я
я
я
я
o ba
garnier
tresemme
e
e
e
e
я
я
я
я
я
я
я
я
я
я
я
я
я
я
магнит
россия
я
я
я
dove
friskies
e
e
курган
магнит
я
fa
fa
fa
ritter sport
ritter sport
ritter sport
e
я
e
радость вкуса
я
я
я
я
e
я
я
garnier
kamis
я
глобус
ной
ной
camay
dove
friskies
friskies
friskies
friskies
friskies
friskies
e
e
я
я
e
я
milka
oreo
чудо
я
я
axe
axe
camay
я
persil
я
fa
я
maggi
e
я
я
kinder
la fresh
магнит
ромашка
я
я
армянский
я
я
сады придонья
e
я
я
kamis
балтика
dove
я
я
магнит
e
e
я
e
zewa
я
я
я
e
domestos
domestos
domestos
domestos
dove
e
ласка
e
schauma
ласка
e
la fresh
ной
чистая линия
tuborg
майский
магнит
я
ной
я
e
я
aro
coca-cola
nestle
магнит
ной
floresan
я
цезарь
gallina blanca
pepsi
la fresh
president
e
e
e
garnier
garnier
garnier
meine liebe
papia
perfect mousse
я
ной
байкал
я
глобус
глобус
я
глобус
глобус
я
я
я
глобус
я
я
я
я
e
я
я
я
магнит
я
я
я
я
я
я
я
я
camay

я
я
e
балтика
bella
la fresh
e
e
я
garnier
garnier
triol
я
я
увелка
фрутоняня
я
e
юбилейное
carlsberg
я
я
я
я
e
я
ушастый нянь
ной
я
e
я
я
я
я
friskies
friskies
e
ной
e
e
e
лента
coca-cola
e
я
лента
я
я
я
я
я
e
e
e
e
la fresh
я
я
магнит
drive me
nestea
pringles
я
агуша
я
магнит
яшкино
e
россия
я
я
я
e
chappi
vernel
e
я
e
магнит
я
я
e
e
vernel
я
я
e
persil
fa
whiskas
whiskas
e
e
я
friskies
dove
e
я
я
ной
ной
e
365
я
шармэль
я
я
365
я
я
ola!
я
большая кружка
большая кружка
ной
я
окей
я
e
e
я
я
магнит
магнит
я
я
лента
e
e
e
я
россия
bic
я
радость вкуса
лента
я
я
лента
лента
e
я
папа может
fa
белый город
экомилк
тысяча озер
e
я
e
свобода
e
e
я
e
e
я
чистая линия
я
e
я
свитлогорье
савушкин
vernel
e
fa
я
e
я
я
вкуснотеево
окей
president
ной
я
я
ной
окей
calve
хлебный дом
e
я
я
россия
365 дней
365 дней
чудо
la fresh
e
майский
майский
я
e
я
я
president
ласка
я
я
e
bref
россия
я
zewa
e
я
я
магнит
тема
я
магнит
чистая линия
absolut
la fresh
e
e
я
я
e
я
ной
я
я
я
я
мжк
чудо
e
e
добрый
parmalat
o 

...

glade
e
я
я
умница
я
я
я
ld
ld
я
я
e
e
milka
я
e
nescafe
добрый
магнит
я
коровка из кореновки
магнит
e
e
магнит
я
e
я
я
e
garnier
kitekat
palmolive
магнит
я
я
e
магнит
colgate
я
я
я
nuts
zollider
e
я
ной
gourmet
e
ld
я
danone
jacobs
я
магнит
я
я
я
e
я
чудо
барни
я
я
я
я
я
я
я
я
я
я
e
я
я
я
я
e
я
я
урожай
я
увелка
snickers
мажитэль
я
я
я
я
я
я
магнит
русский аппетит
я
я
я
e
e
la fresh
gold mine beer
я
aro
я
гейзер
e
vernel
e
черемушки
e
я
я
я
danone
лента
лента
e
я
я
e
шебекинские
я
я
главпродукт
я
affligem
affligem
любимый
e
лента
bella
maggi
петелинка
я
я
я
я
e
балтийский берег
магнит
российский
я
я
e
miller
атяшево
я
россия
я
любимый
любимый
магнит
я
чистая линия
la fresh
я
я
балтика
alpen gold
я
я
я
я
я
я
я
я
троекурово
троекурово
dr.oetker
я
я
я
мжк
я
granmulino
аленка
я
я
магнит
ряба
я
la fresh
bref
я
я
россия
я
aos
friskies
я
365
friskies
actimel
я
магнит
магнит
я
e
я
чистая линия
я
e
aura
bella
dove
glade
bella
dove
e
garnier
galbani
ной
я
e
добрый
село зеленое
я
тонус
365
я
я
я

фруате
окей
я
я
fa
mdd
mdd
mdd
e
я
я
я
e
я
e
e
e
e
я
миф
я
e
магнит
я
я
365
я
я
россия
я
магнит
я
я
я
bella
bella
maybelline
e
я
я
я
я
папа может
я
я
я
arko
365 дней
мартин
365 дней
365 дней
365 дней
я
я
я
e
meine liebe
fa
greenfield
jacobs
la fresh
e
e
я
я
любимый
магнит
сады придонья
я
e
e
увелка
шебекинские
шебекинские
я
e
e
ной
я
магнит
ряба
я
васаби
o ba
chupa chups
я
coca-cola
e
e
e
e
я
я
я
e
я
я
я
я
ной
choco boy
меридиан
простоквашино
я
garnier
e
e
село зеленое
я
лента
e
я
добрый
я
мягкий знак
я
e
я
я
я
я
makfa
глазовская
сладкие истории
я
я
я
friskies
snickers
балтика
балтика
e
veet
я
лента
danone
я
я
federici
makfa
dolce albero
e
лента
лента
e
e
e
dove
я
e
e
365 дней
я
я
я
я
365 дней
я
я
я
garnier
e
e
я
я
e
e
я
e
la fresh
makfa
я
магнит
шебекинские
airwick
bella
я
e
secret
я
e
e
greenfield
я
я
я
ной
я
я
я
меридиан
я
я
магнит
я
я
российский
увелка
увелка
шебекинские
шебекинские
я
магнит
фруате
aura
carefree
dove
я
kind
я
я
брест-литовск
я
я
365
я
я
любимый
e
e
e
e
e
аморе
я
ма

я
я
я
я
тема
я
palmolive
palmolive
e
e
persil
schauma
я
я
магнит
россия
кухмастер
северная гавань
ной
лента
лента
лента
россия
россия
я
я
friskies
friskies
я
магнит
я
я
я
магнит
я
я
e
я
я
я
e
e
e
я
глобус
rocs
sensodyne
ленинградское
я
россия
e
bella
я
я
lady speed stick
lady speed stick
я
glade
e
я
я
e
я
я
я
я
я
букет
cif
я
я
элита
я
я
ряба
мартин
царская
e
я
e
я
я
e
я
я
я
e
я
сады придонья
e
e
мажитэль
я
я
e
e
я
perfect fit
я
я
я
я
e
e
polaris
sanfor
урожай
e
я
365
e
e
axe
chupa chups
я
мартин
магнит
я
я
bud
bref
cif
garnier
johnsons
e
россия
я
я
я
e
я
la fresh
la fresh
la fresh
e
я
я
я
я
я
урожай
я
я
я
я
я
я
я
я
я
я
home story
home story
я
я
e
johnsons
e
johnsons
e
я
я
pepsi
лента
я
я
e
ной
papia
лента
365 дней
365 дней
я
я
e
bella
e
e
e
e
ld
e
e
e
ной
vernel
ной
я
я
я
я
чистая линия
fa
я
lacalut
я
я
я
e
я
la fresh
la fresh
e
mennen
pril
я
ной
лента
makfa
e
лента
kotanyi
лента
лента
я
я
я
я
я
nan
я
я
axe
я
я
я
я
я
я
я
la fresh
я
я
я
магнит
россия
я
e
e
я
я
я
ной
я
я
магнит
фругурт
р

магнит
я
e
e
fa
purina one
я
растишка
я
я
я
я
я
lady speed stick
я
я
я
я
армянский
365
e
e
e
e
e
e
e
e
ной
я
магнит
магнит
e
airwick
e
e
чистая линия
philips
я
я
я
я
vici
бабаевский
бабаевский
я
ежк
я
я
la fresh
bref
choco pie
e
e
алексеевское
я
ласка
я
я
ряба
свитлогорье
свитлогорье
e
danone
veet
e
glade
я
я
магнит
я
я
бонди
я
любимый
я
я
e
я
milka
la fresh
e
e
я
я
барни
я
gold mine beer
белый медведь
я
магнит
axe
лента
arla
майский
магнит
магнит
я
nan
gallina blanca
makfa
makfa
я
я
я
я
я
юбилейное
юбилейное
fa
e
e
я
я
я
maggi
добрый
gerber
cif
dove
dove
e
kleenex
я
e
e
snickers
увелка
я
я
я
я
я
я
я
я
я
я
я
я
я
arla
danone
я
я
e
я
московский провансаль
я
philips
filippo berio
я
я
я
я
я
airwick
airwick
domestos
аморе
глобус
я
я
глобус
coca-cola
аленка
увелка
я
я
я
e
e
я
e
papia
e
e
e
e
лента
e
bonvida
я
я
магнит
persil
я
я
я
я
я
окей
я
я
fa
я
окское
e
e
e
e
la fresh
мартин
я
я
sheba
я
я
я
я
e
axe
alpen gold
любимый
kamis
kamis
я
я
я
магнит
радость вкуса
я
я
я
я
я
я
я
я
я
я
ромашка
я
sc

любимый
e
ной
я
я
я
я
persil
я
schauma
kitekat
майский
магнит
я
dove
dove
я
friskies
friskies
лента
я
лента
365 дней
makfa
я
я
makfa
e
makfa
лента
maggi
я
папа может
papia
e
e
e
я
e
e
окей
окей
я
lady speed stick
jacobs
я
ной
я
e
airwick
glade
gourmet
gourmet
gourmet
gourmet
gourmet
кухмастер
ной
bic
bic
я
я
e
сиртаки
ной
e
snickers
я
garnier
garnier
я
e
я
я
я
я
я
я
я
магнит
я
россия
я
я
я
dove
dove
persil
ласка
чистая линия
dove
dove
persil
ласка
чистая линия
e
я
коровка из кореновки
я
наша маша
пиканта
магнит
я
увелка
я
fa
магнит
я
я
академия комфорта
ной
я
ld
365
carefree
я
я
hochland
whiskas
я
я
магнит
малютка
papia
e
я
365
losk
e
я
fa
e
e
lays
e
e
miller
белый медведь
магнит
я
я
я
я
я
магнит
я
e
лента
e
лента
лента
я
я
e
я
arla
я
zewa
морозко
морозко
e
bref
я
ной
e
fa
пятерочка
365
я
e
мягкий знак
e
мартин
россия
я
любимый
я
я
я
я
морозко
сыробогатов
я
я
наша маша
армянский
e
e
e
ameria
makfa
я
россия
clear
fa
garnier
я
я
магнит
я
миф
bref
maybelline
я
e
я
papia
чистая линия
fa
fa

Получилось, что большое количество ячеек заполнилось буквами "е" и "я", хотя бренды в данных ячейках явно не таких названий. Посмотрим какое количество записей в столбце Бренд осталось пустым, или имеет значение nan, или значение "е", или значение "я".

In [9]:
Cashback_Nans = pd.concat([pd.DataFrame([[i,BrendName[i], Cashback.loc[i, 'Товар']]], 
                                       columns={'Index','Товар','Бренд'}) 
                           for i in np.array(Cashback.query('Бренд != Бренд')['Бренд'].index) 
                           if (BrendName[i] != BrendName[i]) | (BrendName[i] == 'nan') | (BrendName[i] == 'я') 
                           | (BrendName[i] == 'е')])
Cashback_Nans

,Товар,Бренд,Index
0,2,NaN,salton губка/гл кож с дозатор(чер)(шанхай продак
0,3,NaN,soft ватные палочки 200шт пластиковый стакан(кот
0,9,NaN,здоровый стиль мюсли тропические 2
0,13,NaN,с пылу с жару блинчики с творогом
0,22,NaN,томаты 600г
...,...,...,...
0,309687,NaN,торт мирэль прага. 600г
0,309689,NaN,хлеб каравай столовый формовой
0,309690,NaN,чай черный листовой принцесса
0,309695,NaN,<камей>камей мыло туалетное мадемуазель 85гр


Осталось 84 949 пропущенных значений из 178 495. Выгрузим оставшиеся товары и посмотрим с помощбю Excel каких брендов нам не хватает.

In [ ]:
Cashback_Nans.to_csv('Бренд.csv')

Таким образом выделили еще порядка 400 брендов добавим их в первый список брендов.

In [10]:
NewBrends = pd.read_excel('Новые Бренды.xlsx', header=None)
df = NewBrends.iloc[:, 0].tolist()
listNan1 = '|'.join(list(map(str,df)))
list_NanPlus = listNan + listNan1

Еще раз заполним записи столбца Бренд с помощью цикла for и нового списка брендов

In [11]:
chk_pat = '(?:{})'.format(list_NanPlus)
BrendName = Cashback.loc[:,'Бренд']
for i in np.array(Cashback.query('Бренд != Бренд')['Бренд'].index):
    if bool(re.search(chk_pat, Cashback.loc[i,'Товар'], flags=re.I)):
        BrendName[i] = re.search(chk_pat, Cashback.loc[i,'Товар'], flags=re.I).group(0)
        print(BrendName[i])

<ipython-input-11-21e165dba0b5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BrendName[i] = re.search(chk_pat, Cashback.loc[i,'Товар'], flags=re.I).group(0)


salton
здоровый стиль
с пылу с жару
рец.б.агафьи
скинлайт
националь
raid
спецторг
мастер блеск
mirrolla
новый жемчуг
кроха
фа
сибирский мыловар
вк
вологодский хк
вк
лето
несквик
уника
лусетте
лусетте
ив
раптор
ив
дав
мишки в лесу
дав
ив
умка
гл
ив
персил
зева
зева
ив
мультидом
alpica
велес
першинское
мак кофе
чибо голд
велла колор
кит
мамба
аристократ
аристократ
дарница
донна
монарх
хохланд
принглс
r.o.c.s.
фрекен бо
бумеранг
фа
smart
кит
biocos
ив
рец.б.агафьи
вк
куртуа
эгоист
tiamo
йелли
йелли
туал утенок
йелли
мистраль
ив
ив
стейквайн
эгоист
геракл
камис
бархатные ручки
ближние горки
русский продукт
podravka
ариант
зева
pupi
винстон блю
гл
globus
гл
лиронас
вк
ив
ого
knorr
милих пластик
вк
чебаркульский мз
пкк весна
аист
бон пари
бон-пари
лактис
даниссим
гл
mirinda
нарзан
фа
гл
fructis
ооо коломенское
софита
ого
ив
гл
лекс
химпромпроект
белла
ив
гл
белла
молком
symphony
юнилевер
zlaty bazant
аванта
ив
фа
ого
ив
фили
гл
вк
здоровый стиль
дракоша
бондюэль
мамба
гл
оби
воронцовские сух

мастер блеск
fructis
fructis
куес
pupi
pupi
вк
фа
ив
орбит
зингер
гл
фа
тчн!
фили
ого
националь
бархатные ручки
ив
ив
фрекен бо
зеленый луг
вк
санта брем
вк
фа
pupi
вк
вк
здоровый стиль
злато
вк
кубанский молочник
с пылу с жару
фа
хохланд
mirinda
ив
фрекен бо
винстон
волжские просторы
фа
умка
ив
фа
сенеж
мастер блеск
роллтон
гл
аванта
нарзан
зингер
фа
ого
спеленок
ив
юнион
symphony
alpica
белла
ив
снежное лакомство
фа
аванта
гл
alpica
юнилевер
юнилевер
ив
фрекен бо
из села удоево
санта брем
умка
ив
золотой степ
зева
гл
фа
фа
фа
вискас
вискас
вискас
белла
омса
гл
гл
ив
бархатные ручки
гл
волжаночка
вк
нэфис
вк
haribo
злато
фрекен бо
мастер блеск
гл
гл
вк
русский продукт
ив
ив
фа
гл
johnso
domingo
smart
фа
черный жемчуг
черный жемчуг
черный жемчуг
вологжанка
домашний сундук
бархатные ручки
ив
ого
ив
винстон
мегамарт
фа
фа
гл
лекс
вк
вк
вк
бизнесфу
даниссим
ив
сашет
ив
мегамарт
мегамарт
мегамарт
раптор
мегамарт
мегамарт
мегамарт
абрау
гл
вк
domingo
умка
гл
фа
вк
вк
вк
молочный фермер
ролл

гл
гл
мастер блеск
гл
вк
гл
винстон
скинлайт
злато
дав
мастер блеск
ив
верса
вк
котекс
белла
белла
фрекен бо
гл
ого
акконд
фа
аист
доброфлот
ив
777
фрекен бо
ив
палмолив
конти
полимербыт
мамба
дав
ив
шаума
ментос
сплат
омса
зева
вк
фа
фа
мастер блеск
мастер блеск
вк
ментос
гл
лето
фа
фа
ив
вк
вк
гурмэ
гурмэ
гурмэ
гурмэ
волжанка
вк
ого
ого
ого
ив
гл
бархатные ручки
ив
гл
ив
ив
гл
мастер блеск
фа
фа
фа
вк
фа
фа
вк
хохланд
фа
salton
ив
наша корова
smart
аванта
милих пластик
фа
ив
ив
ив
smart
palm
гл
ив
мартика
гл
фа
alpica
ив
ив
fructis
ив
вк
вк
гл
ив
фа
вк
вк
ив
гл
ив
вк
гл
вк
бон-аква
мастер блеск
здоровый стиль
ого
конти
ив
сашет
молочный фермер
русский продукт
кит
винстон
ив
фрекен бо
фрекен бо
smart
юнилевер
домашний сундук
babycomfort
зева
шаума
шаума
вк
гл
фа
мастер блеск
фа
ив
ив
мастер блеск
фрекен бо
лето
гл
великолу
вк
smart
ив
мастер блеск
mirrolla
ив
фа
ив
мастер блеск
дракоша
мастер блеск
вк
гл
smart
ив
гл
ив
ив
персил
умка
globus
наша корова
alpica
аура
фа
фа
фа
фрекен бо
с

эклипс
symphony
даниссим
мастер блеск
фа
лекс
балтком
индилайт
palm
smart
фа
фа
фа
куес
дуру
фа
гл
фа
ив
вк
вк
shamtu
фрекен бо
фрекен бо
семейные секр
фа
вк
вк
гл
фа
гл
гарньер
ив
хохланд
ближние горки
ив
белла
фа
росита
вк
гл
фа
фа
фа
момент
ого
кит
националь
пава пава
фа
наша корова
бархатные ручки
ив
весёлые прилипалы
фа
доместос
ив
дуру
globus
злато
фа
гл
ив
alpica
лекс
белевск
фа
весёлые прилипалы
нарзан
симилак
лекс
знатные
знатные
фа
гл
ив
белла
ессентуки
ив
гл
доброфлот
фа
дарница
fructis
ив
черный жемчуг
даренка
золотой стандарт
7days
воронцовские сухарики
роллтон
вк
ив
palm
мастер блеск
кубанский молочник
националь
фа
момент
мастер блеск
фа
вк
из села удоево
фа
фа
вк
лекс
гл
ив
ив
ив
националь
бондюэль
вк
орбит
зева
фа
вк
молочный фермер
мультидом
умка
злато
гл
националь
фрекен бо
ив
вк
вк
ив
фрекен бо
лето
белла
фа
pupi
вискас
гл
фа
националь
гл
glorix
бархатные ручки
вк
фа
ломтишка
r.o.c.s 
фа
фа
гл
вк
петели
alpica
даниссим
фа
ив
ив
дарница
мастер блеск
аист
белла
фа
виол

...

гл
гл
вк
гл
националь
вог
johnso
бархатные ручки
даниссим
колгейт
мол благодат
мол благодат
мол благодат
ив
pupi
мультидом
черный жемчуг
гл
омса
вк
ив
фрекен бо
гл
шаума
шаума
фа
фа
фа
момент
момент
ив
ив
виста
белла
белла
ив
доместос
ив
garni
вк
ив
ив
фа
фа
фрекен бо
колгейт
дав
ив
умка
фа
ив
гл
белла
ив
ив
принцесса
гл
фа
фа
ив
гл
ив
ив
ив
кит
ив
гл
фрекенбок
черный жемчуг
ив
ив
зева
гл
фа
полимербыт
ив
гл
фр.бок
вк
ив
вк
данон
злато
бархатные ручки
гл
фили
вк
ив
ив
rothmans
гл
черный жемчуг
вк
лето
вк
вк
ого
котекс
лейз
biocos
ив
ив
фа
мастер блеск
ив
белла
ив
ив
гл
гл
фа
доминго
мастер блеск
мастер блеск
мастер блеск
фрекенбок
фа
фа
мастер блеск
гл
белла
вк
фрекен бо
гл
гл
фа
гл
роллтон
symphony
гл
аист
мастер блеск
зингер
alpica
мастер блеск
ого
приоскол
вк
ив
бархатные ручки
аванта
палмолив
палмолив
ив
мастер блеск
вк
скинлайт
роллтон
вк
колгейт
лакалют
ив
ив
аист
котекс
ив
мастер блеск
золотой стандарт
националь
фа
фа
фа
винстон блю
palm
вк
biocos
мастер блеск
бархатные ручки
вк

саф
черный жемчуг
мастер блеск
мастер блеск
ого
аист
акконд
фа
haribo
мартика
весёлые прилипалы
санта брем
raid
аист
мамба
мамба
ив
ив
вк
шаума
роллтон
вк
самсо
скинлайт
вк
новый жемчуг
бон пари
вк
гл
вк
сашет
ив
омса
вк
smart
бархатные ручки
мастер блеск
фрекенбок
фрекен бо
smart
чибо голд
ив
фа
ив
умка
гл
ив
pupi
ломтишка
ого
biocos
гл
smart
аура
дав
дав
мастер блеск
колгейт
вк
ив
туал утенок
гл
biocos
даниссим
даниссим
першинское
оби
аванта
ого
ив
лето
ив
доброфлот
фа
акконд
вк
гл
ив
ив
ив
умка
умка
фа
фа
вискас
вискас
ив
ив
вк
куес
ив
ив
даниссим
даниссим
ив
ив
мастер блеск
ив
фа
мистраль
alpica
мастер блеск
черный жемчуг
гл
comfort
палмолив
гл
фа
фа
фа
фа
фа
alpica
вк
кубанский молочник
ив
ив
fructis
фа
дарница
омса
мастер блеск
фрекен бо
фа
фа
вк
symphony
аванта
умка
knorr
фа
вк
фа
вк
белла
фрекен бо
винстон
аист
мастер блеск
гл
фа
аист
полимербыт
мастер блеск
белла
knorr
knorr
knorr
здоровый стиль
ив
ариант
ариант
ив
винстон
принглс
даниссим
ого
ив
ив
вк
фа
фа
сашет
мастер блеск

вк
фа
кит
фа
гл
семейные секр
вк
almax
черный жемчуг
бон пари
снежный краб
фа
knorr
вк
ив
саф
полимербыт
фа
ив
ив
bonjour
белла
фили
ив
фа
фа
ив
fructis
fructis
ив
весёлые прилипалы
ив
ив
grifon
ив
вк
фа
молком
куес
полимербыт
фили
аист
мастер блеск
фр.бок
данон
данон
фа
ив
золотой шелк
фа
ив
гл
мастер блеск
гл
ив
вк
ив
черный жемчуг
ив
вк
rothmans
domingo
белла
белла
джинн
националь
fructis
ив
черный жемчуг
фа
санта брем
вк
вк
ого
ближние горки
ого
фа
фа
санта брем
ив
смирнов
фа
ив
smart
ив
фили
гл
фа
фа
ив
pupi
момент
гл
дабрович
новый жемчуг
весёлые прилипалы
ив
черный жемчуг
черный жемчуг
черный жемчуг
фа
вк
мастер блеск
вк
куриное царство
гл
акконд
с пылу с жару
фа
фа
кит
вк
вними-сибирь
гл
роллтон
гл
фа
гл
smart
винстон блю
alpica
alpica
мартика
фили
ив
фрекен бо
н/жемчуг
хрустеам
хрустеам
вк
ого
черный жемчуг
smart
гл
черный жемчуг
вк
вк
777
fructis
фрекен бо
ессентуки
ив
винстон
гл
фа
фа
ив
петели
петели
гл
pupi
аист
ив
даниссим
ив
фа
ив
русский продукт
даниссим
санта брем
alpi

фа
мастер блеск
фа
вк
ив
доброфлот
доброфлот
фа
националь
вк
фа
фа
курин царст
ив
ив
мастер блеск
гранд десерт
кит
фа
фа
ив
knorr
knorr
knorr
вк
вк
лакалют
гл
роллтон
ив
гл
из села удоево
индилайт
юнион
аист
с пылу с жару
гл
индилайт
pupi
pupi
ив
вк
777
фр.бок
вк
вк
несквик
ментос
подворье
злато
первым делом
весёлые прилипалы
подружка
ив
умка
аист
ив
гл
гл
гл
ив
ив
бархатные ручки
бархатные ручки
alpica
ив
гл
фрекен бо
мастер блеск
мастер блеск
comfort
фа
ив
гл
вк
ив
фа
туал утенок
гл
ив
хрустеам
вк
гл
фа
фа
белла
белла
мастер блеск
новый жемчуг
ив
ив
fructis
ив
гл
гл
ив
гл
риттер спорт
риттер спорт
риттер спорт
ив
даренка
момент
аист
аист
мастер блеск
принцесса
новый жемчуг
ив
данон
искренне ваш
гл
националь
biocos
вк
ив
babycomfort
гл
фа
вк
вк
киржачский
ив
оби
бархатные ручки
гл
санта брем
фа
ив
вк
конти
зева
вк
вк
гл
alpica
фрекен бо
даниссим
ив
здоровый стиль
русский продукт
аванта
золотой стандарт
золотой стандарт
гл
ив
фа
biocos
мастер блеск
вог
бон пари
националь
ив
зева
полиме

ив
донской табак
фа
ив
вк
вк
ив
омса
абрау
вк
palm
smart
вискас
вк
ив
знатные
фа
вк
мастер блеск
haribo
вк
фа
гл
гл
националь
зеленый луг
оби
гл
фрекен бо
гл
палмолив
палмолив
сплат
вк
alpica
glorix
с пылу с жару
бархатные ручки
гл
ив
принцесса


Поосмотрим какое количество брендов осталось на данном этапе

In [12]:
Cashback_Nans = pd.concat([pd.DataFrame([[i,BrendName[i], Cashback.loc[i, 'Товар']]], 
                                       columns={'Index','Товар','Бренд'}) 
                           for i in np.array(Cashback.query('Бренд != Бренд')['Бренд'].index) 
                           if (BrendName[i] != BrendName[i]) | (BrendName[i] == 'nan') | (BrendName[i] == 'я') 
                           | (BrendName[i] == 'е')])
Cashback_Nans

,Товар,Бренд,Index
0,3,NaN,soft ватные палочки 200шт пластиковый стакан(кот
0,22,NaN,томаты 600г
0,28,NaN,"3406187 наша кор.мол.2,5% пэт 900мл"
0,30,NaN,"3666192 дан.пр.тв.гр.ор-кл.с.5,9%130г"
0,43,NaN,бананы 1кг
...,...,...,...
0,309685,NaN,сочни аладушкин черника 120г
0,309687,NaN,торт мирэль прага. 600г
0,309689,NaN,хлеб каравай столовый формовой
0,309695,NaN,<камей>камей мыло туалетное мадемуазель 85гр


64367 оставшихся значений заполним нулями при помощи метода fillna().

In [13]:
Cashback[Cashback['Бренд'] != Cashback['Бренд']] # проверка, что 64367 записей - Nan

,﻿id чека,Дата и время,ИНН,IDПользователя,Товар,Бренд,Теги,Цена,Кол-во,Сумма,Кэшбэк
3,0000a622f9c8f1821ca549fbcfd47405,21.12.2019 15:15,2310031475,2700449a73d4cb9d8e909c706bf16fbe,soft ватные палочки 200шт пластиковый стакан(кот,NaN,NaN,31.20,1.000,31.20,NaN
22,00011b384dff1067131b3ecaea35382f,07.12.2019 14:09,2310031475,c8142fe20d51c0875ab409128c0a6477,томаты 600г,NaN,NaN,82.90,1.000,82.90,NaN
28,000138502d273274fc30a8d28db41680,21.12.2019 12:51,7825706086,3ea38f64bac275ad45c4ec07025f9086,"3406187 наша кор.мол.2,5% пэт 900мл",NaN,NaN,47.99,1.000,47.99,NaN
30,000138502d273274fc30a8d28db41680,21.12.2019 12:51,7825706086,3ea38f64bac275ad45c4ec07025f9086,"3666192 дан.пр.тв.гр.ор-кл.с.5,9%130г",NaN,NaN,34.99,1.000,34.99,NaN
43,0002a37e0af2f9b8e081119e3e50209d,14.01.2020 20:08,2310031475,22903834207878a922d05c5b7fde4077,бананы 1кг,NaN,продукты; банан; фрукты; овощи/фрукты/ягоды,69.90,0.986,68.92,NaN
...,...,...,...,...,...,...,...,...,...,...,...
309685,5128ca7cab1f2f6b54b5569c3a09ce11,02.11.2019 17:22,7826087713,1bb75e31dc7a21e8b4de5e34d16fb103,сочни аладушкин черника 120г,NaN,NaN,19.99,1.000,19.99,NaN
309687,5128ca7cab1f2f6b54b5569c3a09ce11,02.11.2019 17:22,7826087713,1bb75e31dc7a21e8b4de5e34d16fb103,торт мирэль прага. 600г,NaN,продукты; торт; хлеб и сладости,169.50,1.000,169.50,NaN
309689,5128ca7cab1f2f6b54b5569c3a09ce11,02.11.2019 17:22,7826087713,1bb75e31dc7a21e8b4de5e34d16fb103,хлеб каравай столовый формовой,NaN,хлеб и сладости; хлеб; продукты,32.89,1.000,32.89,NaN
309695,5128dbb3ea9c658c01187063dfd48032,31.01.2020 18:13,5405191698,bb2ef9a68abdf03fe71aeb862d1173af,<камей>камей мыло туалетное мадемуазель 85гр,NaN,NaN,33.49,1.000,33.49,NaN


In [14]:
Cashback['Бренд'] = Cashback['Бренд'].fillna(0)
Cashback[Cashback['Бренд'] != Cashback['Бренд']]

,﻿id чека,Дата и время,ИНН,IDПользователя,Товар,Бренд,Теги,Цена,Кол-во,Сумма,Кэшбэк


Посмотрим какие товары имеют отсутствующие значения в стобцах Сумма, Цена, Кол-во

In [15]:
Cashback[Cashback['Цена'] != Cashback['Цена']]

,﻿id чека,Дата и время,ИНН,IDПользователя,Товар,Бренд,Теги,Цена,Кол-во,Сумма,Кэшбэк
2398,0097e0d2a7a3adec87913cee91110f97,25.01.2020 14:19,"1,663E+11",fd5653334f813e4e2a7546e63384fee8,**удобрение агрикола универсальное комплексное д,ив,NaN,NaN,NaN,NaN,NaN
2400,0097e0d2a7a3adec87913cee91110f97,25.01.2020 14:19,"1,663E+11",fd5653334f813e4e2a7546e63384fee8,олвейз прокладки гиг. ежедневн. незаметная защит,я,NaN,NaN,NaN,NaN,NaN
6041,018e97fb214d360af3ca5bbced3ea7cb,22.01.2020 17:09,"1,65061E+11",b6a84fa6ef1a3cadcd5f442f02a2512b,олвейз прокладки гиг. ежедневн. незаметная защит,я,NaN,NaN,NaN,NaN,NaN
14537,03d3ab2b0fba7616378d8b03496b8550,11.11.2019 16:27,"1,65032E+11",4be0b47f24bbb956ca25418972fbc614,*фейри ср-во д/посуды чайное дерево и мята 450 м,я,NaN,NaN,NaN,NaN,NaN
24055,0642206ed4abd1a7fe57f128b46c9291,25.11.2019 19:36,"1,65026E+11",ca8daf78d673d671cbdcc3af1d52783d,*ленор кондиционер концентрир. д/белья детский 2,я,NaN,NaN,NaN,NaN,NaN
63954,10f5c85b77dfbdc4f4667c3f23c12b03,19.11.2019 15:30,"1,65032E+11",b01d9dc7bb784152d6d32056cc2b7b89,колгейт древние секреты зуб. паста укрепление эм,колгейт,NaN,NaN,NaN,NaN,NaN
63955,10f5c85b77dfbdc4f4667c3f23c12b03,19.11.2019 15:30,"1,65032E+11",b01d9dc7bb784152d6d32056cc2b7b89,колгейт зуб. щетка массажная средняя 1 шт. +1 шт,я,NaN,NaN,NaN,NaN,NaN
63959,10f5c85b77dfbdc4f4667c3f23c12b03,19.11.2019 15:30,"1,65032E+11",b01d9dc7bb784152d6d32056cc2b7b89,экзэ гель д/бритья д/ч/кожи успокаив. сенсетив 2,я,NaN,NaN,NaN,NaN,NaN
76826,1479846b8dd1138d66ca494cc59ca8e1,26.01.2020 17:02,"1,663E+11",dd6c8c7f01c0b8f2cdda68eb50b82595,**пантин бальзам д/волос укрепление и блеск д/то,0,NaN,NaN,NaN,NaN,NaN
94198,194f3f8ca33cc0fed3535e2e05a1a7db,16.01.2020 12:23,"1,65061E+11",8bdec293d35bb09029edd6c31f663911,олвейз прокладки гиг. ежедневн. незаметная защит,я,NaN,NaN,NaN,NaN,NaN


Так как товары из списка не относятся к категории продукты, то скорей всего данные товары покупались поштучно и столи около 100 рублей. Поэтому отсутствующие значения полей Сумма заполним - 100, Цена - 100, Кол-во - 1, а Тэги - 'None' и Кэшбэк заполним нулями, так как отсутствие записи в поле кэшбэк скорей всего означет отсутствие кэшбэка.

In [16]:
Cashback['Цена'] = Cashback['Цена'].fillna(100)
Cashback['Сумма'] = Cashback['Сумма'].fillna(100)
Cashback['Кол-во'] = Cashback['Кол-во'].fillna(1)
Cashback['Теги'] = Cashback['Теги'].fillna('None')
Cashback['Кэшбэк'] = Cashback['Кэшбэк'].fillna(0)

Проверим, что больше нет отсутствующих значений в датасете

In [17]:
Cashback_NanNew = pd.concat([pd.DataFrame([[Cashback.columns[i], Cashback[Cashback.columns[i]].isna().sum()]], 
                                       columns={'Column','Nan Quantity'}) for i in range(5,11)])
Cashback_NanNew.set_index('Column', inplace=True)
Cashback_NanNew

,Nan Quantity
Column,
Бренд,0
Теги,0
Цена,0
Кол-во,0
Сумма,0
Кэшбэк,0


In [18]:
cols = ['Бренд', 'Теги', 'Цена', 'Кол-во', 'Сумма', 'Кэшбэк']
Cashback[cols] = Cashback[cols].fillna(Cashback.mode().iloc[0])

In [19]:
Cashback.mode().iloc[0]

﻿id чека          437ece69380b949b66b27d392c7b70b3
Дата и время                      26.12.2019 13:41
ИНН                                     2310031475
IDПользователя    3e93fb9eeddf9dd6cf93c5be82b3e374
Товар                           пакет-майка магнит
Бренд                                            0
Теги                                          None
Цена                                           199
Кол-во                                           1
Сумма                                          199
Кэшбэк                                           0
Name: 0, dtype: object

Сгруппируем значения по одному чеку  и сформируем лист по сгруппированым значениям:

In [20]:
CashbackGrouped = Cashback.groupby('﻿id чека')["Теги"].unique()

In [18]:
CashbackTuple = list(map(tuple, CashbackGrouped))

Используем Аpriori с подготовленными данными

In [9]:
from efficient_apriori import apriori

In [19]:
association_rules = apriori(CashbackTuple, min_support=0.007,
                            min_confidence=0.5)

In [20]:
print(association_rules[1]) # печатаем сами правила


[{личная гигиена; красота и здоровье; прокладки} -> {упаковка; пакеты; прочее}, {прокладки; личная гигиена; красота и здоровье} -> {упаковка; пакеты; прочее}, {красота и здоровье; гигиена волос; шампунь для волос} -> {упаковка; пакеты; прочее}, {красота и здоровье; дезодорант/антиперсперант; личная гигиена} -> {упаковка; пакеты; прочее}, {продукты; банан; фрукты; овощи/фрукты/ягоды} -> {упаковка; пакеты; прочее}, {бакалея; сахар; продукты} -> {упаковка; пакеты; прочее}, {красота и здоровье; прокладки; личная гигиена} -> {упаковка; пакеты; прочее}, {красота и здоровье; личная гигиена; прокладки} -> {упаковка; пакеты; прочее}, {уход за полостью рта; красота и здоровье; зубная паста} -> {упаковка; пакеты; прочее}, {дезодорант/антиперсперант; личная гигиена; красота и здоровье} -> {упаковка; пакеты; прочее}, {красота и здоровье; шампунь для волос; гигиена волос} -> {упаковка; пакеты; прочее}, {хлеб и сладости; хлеб; продукты} -> {упаковка; пакеты; прочее}, {уход за полостью рта; зубная пас

In [21]:
print(association_rules[0])

{1: {('упаковка; пакеты; прочее',): 30585, ('личная гигиена; красота и здоровье; прокладки',): 2410, ('прокладки; личная гигиена; красота и здоровье',): 2484, ('уход за полостью рта; зубная паста; красота и здоровье',): 760, ('красота и здоровье; гигиена волос; шампунь для волос',): 1043, ('красота и здоровье; дезодорант/антиперсперант; личная гигиена',): 521, ('продукты; банан; фрукты; овощи/фрукты/ягоды',): 699, ('дезодорант/антиперсперант; личная гигиена; красота и здоровье',): 504, ('; ',): 550, ('красота и здоровье; личная гигиена; дезодорант/антиперсперант',): 707, ('бакалея; сахар; продукты',): 400, ('красота и здоровье; прокладки; личная гигиена',): 1281, ('красота и здоровье; зубная паста; уход за полостью рта',): 639, ('кондиционер для белья; хозяйственные товары; уход за бельем',): 1109, ('красота и здоровье; личная гигиена; прокладки',): 1261, ('уход за полостью рта; красота и здоровье; зубная паста',): 613, ('красота и здоровье; шампунь для волос; гигиена волос',): 1328, (

In [22]:
for item in association_rules:
    pair = item[1]
    print(pair)

{('упаковка; пакеты; прочее',): 30585, ('личная гигиена; красота и здоровье; прокладки',): 2410, ('прокладки; личная гигиена; красота и здоровье',): 2484, ('уход за полостью рта; зубная паста; красота и здоровье',): 760, ('красота и здоровье; гигиена волос; шампунь для волос',): 1043, ('красота и здоровье; дезодорант/антиперсперант; личная гигиена',): 521, ('продукты; банан; фрукты; овощи/фрукты/ягоды',): 699, ('дезодорант/антиперсперант; личная гигиена; красота и здоровье',): 504, ('; ',): 550, ('красота и здоровье; личная гигиена; дезодорант/антиперсперант',): 707, ('бакалея; сахар; продукты',): 400, ('красота и здоровье; прокладки; личная гигиена',): 1281, ('красота и здоровье; зубная паста; уход за полостью рта',): 639, ('кондиционер для белья; хозяйственные товары; уход за бельем',): 1109, ('красота и здоровье; личная гигиена; прокладки',): 1261, ('уход за полостью рта; красота и здоровье; зубная паста',): 613, ('красота и здоровье; шампунь для волос; гигиена волос',): 1328, ('хле

#  Поиск товаров P&G в чеках покупателей

Необходимо сгенерировать DataFrame, в котором присутствует количество каждого из брендов P&G в чеках покупателей.
Для этого создадим пользовательскую функцию, подсчитывающую количество товаров определенного бренда, который подается на вход данной функции

In [21]:
def count_brand(brand_name):
    found_rows = Cashback['Бренд'].str.contains(brand_name).sum()
    return found_rows

In [22]:
BrendsPG = pd.read_excel('Бренды PG.xlsx', header=None)
df = BrendsPG.iloc[:, 0].tolist()

С помощью функции concat получим необходимый DataFrame.

In [27]:
Cashback_PG = pd.concat([pd.DataFrame([[BrendsPG.iat[i,0], count_brand(BrendsPG.iat[i,0])]], 
                                      columns={'Бренд','Кол-во'}) for i in range(0, len(BrendsPG))],ignore_index=True)
#Cashback_PG.set_index('Index')
Cashback_PG

,Бренд,Кол-во
0,pampers,3505
1,tide,1960
2,миф,345
3,lenor,5207
4,олвейз,2
5,always,10737
6,tampax,228
7,naturella,2989
8,discreet,6250
9,oral-b,1116


Бренды олвейз и always один и тот же, объединим эти две строчки.

In [28]:
Cashback_PG=Cashback_PG.drop(4,axis=0)

In [30]:
Cashback_PG.loc[5,'Кол-во'] = 10739

Окончательный DataFrame

In [31]:
Cashback_PG

,Бренд,Кол-во
0,pampers,3505
1,tide,1960
2,миф,345
3,lenor,5207
5,always,10739
6,tampax,228
7,naturella,2989
8,discreet,6250
9,oral-b,1116
10,gillette,5721


In [ ]:
Cashback_PG.plot.barh(x='Бренд', y='Кол-во')